# Production Planning Problem
* Linear Programming


In [33]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from collections import OrderedDict

## Set decision variables

In [34]:
# Model
m = gp.Model("PB")

# Decision variables for the regular, extra production, and stock by month
x1_1 = m.addVar(name="Normal_Prod1")
x1_2 = m.addVar(name="Extra_Prod1")
x2_1 = m.addVar(name="Normal_Prod2")
x2_2 = m.addVar(name="Extra_Prod2")
x3_1 = m.addVar(name="Normal_Prod3")
x3_2 = m.addVar(name="Extra_Prod3")
x4_1 = m.addVar(name="Normal_Prod4")
x4_2 = m.addVar(name="Extra_Prod4")
s1 = m.addVar(name="stock1")
s2 = m.addVar(name="stock2")
s3 = m.addVar(name="stock3")

## Define an objective function and constraints

In [35]:
# Formulation 

#### Objective Function ####
# The objective is to minimize the total cost
obj = 440*(x1_1+x2_1+x3_1+x4_1) + 700*(x1_2+x2_2+x3_2+x4_2)+5*(s1+s2+s3)

m.setObjective(obj, GRB.MINIMIZE)

#### Constraints ####
# Regular production <= max regular production   
con1 = m.addConstr(x1_1 <= 140, name = "max_normProd1")
con2 = m.addConstr(x2_1 <= 150, name = "max_normProd2")
con3 = m.addConstr(x3_1 <= 140, name = "max_normProd3")
con4 = m.addConstr(x4_1 <= 160, name = "max_normProd4")

# Extra production <= max extra production
con5 = m.addConstr(x1_2 <= 50, name = "max_extraProd1")
con6 = m.addConstr(x2_2 <= 75, name = "max_extraProd2")
con7 = m.addConstr(x3_2 <= 70, name = "max_extraProd3")
con8 = m.addConstr(x4_2 <= 80, name = "max_extraProd4")

# stock(i-1) + production(i) - stock(i) = demand(i)
con9 = m.addConstr(x1_1+x1_2-s1 == 120, name = "meet_demand1")
con10 = m.addConstr(s1+x2_1+x2_2-s2 == 160, name = "meet_demand2")
con11 = m.addConstr(s2+x3_1+x3_2-s3 == 300, name = "meet_demand3")
con12 = m.addConstr(s3+x4_1+x4_2 == 200, name = "meet_demand4")

# Max capacity of warehouse 
con13 = m.addConstr(s1 <= 100, name = "max_cap1")
con14 = m.addConstr(s2 <= 100, name = "max_cap2")
con15 = m.addConstr(s3 <= 100, name = "max_cap3")

# Normal production of each month >= 10% of the total production of the first three months
con16 = m.addConstr(x1_1 >= 0.1*(x1_1+x1_2+x2_1+x2_2+x3_1+x3_2), name = "balanced_prod1")
con17 = m.addConstr(x2_1 >= 0.1*(x1_1+x1_2+x2_1+x2_2+x3_1+x3_2), name = "balanced_prod2")
con18 = m.addConstr(x3_1 >= 0.1*(x1_1+x1_2+x2_1+x2_2+x3_1+x3_2), name = "balanced_prod3")
con19 = m.addConstr(x4_1 >= 0.1*(x1_1+x1_2+x2_1+x2_2+x3_1+x3_2), name = "balanced_prod4")

# Non-negativity constraints
con20 = m.addConstr(x1_1 >= 0, name = "NonNegative1_1")
con21 = m.addConstr(x1_2 >= 0, name = "NonNegative1_2")
con22 = m.addConstr(x2_1 >= 0, name = "NonNegative2_1") 
con23 = m.addConstr(x2_2 >= 0, name = "NonNegative2_2") 
con24 = m.addConstr(x3_1 >= 0, name = "NonNegative3_1") 
con25 = m.addConstr(x3_2 >= 0, name = "NonNegative3_2") 
con26 = m.addConstr(x4_1 >= 0, name = "NonNegative4_1") 
con27 = m.addConstr(x4_2 >= 0, name = "NonNegative4_2") 
con28 = m.addConstr(s1 >= 0, name = "NonNegative_s1")
con29 = m.addConstr(s2 >= 0, name = "NonNegative_s2")
con30 = m.addConstr(s3 >= 0, name = "NonNegative_s3")

# Solve
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 30 rows, 11 columns and 61 nonzeros
Model fingerprint: 0x09eca39d
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [5e+00, 7e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 3e+02]
Presolve removed 27 rows and 3 columns
Presolve time: 0.00s
Presolved: 3 rows, 8 columns, 10 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.4125935e+05   4.191750e+01   0.000000e+00      0s
       3    3.9317500e+05   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds
Optimal objective  3.931750000e+05


## Optimal value with optimal variables

In [36]:
# Print optimal value of the objective function
print('\nTotal cost to produce rice over 4 months: $ %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))


Total cost to produce rice over 4 months: $ 393175

Decision variables:
Normal_Prod1 = 140
Extra_Prod1 = 5
Normal_Prod2 = 150
Extra_Prod2 = 75
Normal_Prod3 = 140
Extra_Prod3 = 70
Normal_Prod4 = 160
Extra_Prod4 = 40
stock1 = 25
stock2 = 90
stock3 = 0


## Sensitivity Analysis

In [31]:
# Create table for decision variables' sensitivity analysis
decision_var = OrderedDict([
    ('Name', ['x1_1', 'x1_2', 'x2_1', 'x2_2', 'x3_1', 'x3_2', 'x4_1', 'x4_2', 's1', 's2', 's3']),
    ('Final Value', [x1_1.x, x1_2.x, x2_1.x, x2_2.x, x3_1.x, x3_2.x, x4_1.x, x4_2.x, s1.x, s2.x, s3.x]),
    ('Reduced Cost', [x1_1.RC, x1_2.RC, x2_1.RC, x2_2.RC, x3_1.RC, x3_2.RC, x4_1.RC, x4_2.RC, s1.RC, s2.RC, s3.RC]),
    ('Obj Coeff', [440,700,440,700,440,700,440,700,5,5,5]),
    ('Upper Range', [x1_1.SAObjUp, x1_2.SAObjUp, x2_1.SAObjUp, x2_2.SAObjUp, 
                     x3_1.SAObjUp, x3_2.SAObjUp, x4_1.SAObjUp, x4_2.SAObjUp,
                     s1.SAObjUp, s2.SAObjUp, s3.SAObjUp]),
    ('Lower Range', [x1_1.SAObjLow, x1_2.SAObjLow, x2_1.SAObjLow, x2_2.SAObjLow, 
                     x3_1.SAObjLow, x3_2.SAObjLow, x4_1.SAObjLow, x4_2.SAObjLow, 
                     s1.SAObjLow, s2.SAObjLow, s3.SAObjLow])
])


# Print sensitivity analysis tables for decision variables and constraints
print('\n')
print(pd.DataFrame.from_dict(decision_var))




    Name  Final Value  Reduced Cost  Obj Coeff  Upper Range  Lower Range
0   x1_1        140.0           0.0        440        700.0         -inf
1   x1_2          5.0           0.0        700          inf        695.0
2   x2_1        150.0           0.0        440        705.0         -inf
3   x2_2         75.0           0.0        700        705.0         -inf
4   x3_1        140.0           0.0        440        710.0         -inf
5   x3_2         70.0           0.0        700        710.0         -inf
6   x4_1        160.0           0.0        440        700.0         -inf
7   x4_2         40.0           0.0        700        715.0        440.0
8     s1         25.0           0.0          5          inf          0.0
9     s2         90.0           0.0          5          inf         -5.0
10    s3          0.0          15.0          5          inf        -10.0
